In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import batch_norm
from tensorflow.examples.tutorials.mnist import input_data
from skimage import transform
import warnings

In [2]:
warnings.simplefilter('ignore')
mnist = input_data.read_data_sets("data")
tf.reset_default_graph()

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [12]:
tf.reset_default_graph()
image_size=28
n_inputs = 28*28 # MNIST
n_hidden1 = 700
n_hidden2 = 500
n_hidden3=400
n_hidden4=300
n_hidden5=200
n_hidden6=100
n_hidden7=20
n_outputs = 10


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
        'is_training': is_training,
        'decay': 0.99,
        'updates_collections': None
        }



In [13]:
with tf.name_scope("dnn"):
#    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
#    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
#    logits = fully_connected(hidden2, n_outputs, scope="outputs",
#                             activation_fn=None)
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden3 = fully_connected(hidden2, n_hidden3, scope="hidden3",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden4 = fully_connected(hidden3, n_hidden4, scope="hidden4",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden5 = fully_connected(hidden4, n_hidden5, scope="hidden5",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden6 = fully_connected(hidden5, n_hidden6, scope="hidden6",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    hidden7 = fully_connected(hidden6, n_hidden7, scope="hidden7",
                              normalizer_fn=batch_norm, normalizer_params=bn_params)
    logits = fully_connected(hidden7, n_outputs, activation_fn=None,scope="outputs",
                             normalizer_fn=batch_norm, normalizer_params=bn_params)


In [14]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [15]:
learning_rate = 0.001

with tf.name_scope("train"):
#    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [16]:
init = tf.global_variables_initializer()
n_epochs = 100000
batch_size = 3000

In [17]:
n=0
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n=0
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            RR,acc_train=sess.run([training_op, accuracy], feed_dict={is_training: True,X: X_batch, y: y_batch})
            n+=acc_train
        acc_test = accuracy.eval(feed_dict={is_training:False,
                                            X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train accuracy:", n/(mnist.train.num_examples // batch_size), 
              "Test accuracy:", acc_test)
        

0 Train accuracy: 0.803814854059 Test accuracy: 0.8678
1 Train accuracy: 0.950703766611 Test accuracy: 0.9435
2 Train accuracy: 0.971203757657 Test accuracy: 0.9539
3 Train accuracy: 0.982388916943 Test accuracy: 0.956
4 Train accuracy: 0.988425970078 Test accuracy: 0.9563
5 Train accuracy: 0.992722292741 Test accuracy: 0.9494
6 Train accuracy: 0.995444526275 Test accuracy: 0.949
7 Train accuracy: 0.997092700667 Test accuracy: 0.9502
8 Train accuracy: 0.998092687792 Test accuracy: 0.9566
9 Train accuracy: 0.998463074366 Test accuracy: 0.9629
10 Train accuracy: 0.999037133323 Test accuracy: 0.9687
11 Train accuracy: 0.999111202028 Test accuracy: 0.9709
12 Train accuracy: 0.999388986164 Test accuracy: 0.9748
13 Train accuracy: 0.999426003959 Test accuracy: 0.9745
14 Train accuracy: 0.999518622955 Test accuracy: 0.9773
15 Train accuracy: 0.999740855561 Test accuracy: 0.9787
16 Train accuracy: 0.999666776922 Test accuracy: 0.9789
17 Train accuracy: 0.99979640709 Test accuracy: 0.9804
18 Tr

148 Train accuracy: 1.00000011921 Test accuracy: 0.9813
149 Train accuracy: 1.00000011921 Test accuracy: 0.9815


KeyboardInterrupt: 

In [8]:
tf.reset_default_graph()